In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields)

In [3]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [4]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  843.0


In [5]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  223.0


In [6]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [7]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 8451 personas en los últimos 7 días 

Un positivo PCR cada 4634 personas en los últimos 14 días


In [8]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [9]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [10]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [11]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [12]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] > 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,843.0,126.0,223.0,510743.0,24.669942,43.661881,32.0
Condado-Campiña,257.0,61.0,92.0,155057.0,39.340372,59.333019,19.0
Huelva-Costa,434.0,54.0,101.0,288115.0,18.742516,35.055447,11.0
Bollullos Par del Condado,23.0,11.0,12.0,14272.0,77.073991,84.080717,5.0
Palma del Condado (La),29.0,9.0,11.0,10761.0,83.635350,102.220983,5.0
Lepe,59.0,8.0,21.0,27431.0,29.164084,76.555722,4.0
Huelva (capital),223.0,17.0,31.0,143663.0,11.833249,21.578277,3.0
Villalba del Alcor,30.0,17.0,28.0,3338.0,509.286998,838.825644,3.0
Escacena del Campo,11.0,7.0,10.0,2284.0,306.479860,437.828371,2.0


In [13]:
# Save to csv
tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)

In [14]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Villalba del Alcor,30.0,17.0,28.0,3338.0,509.286998,838.825644,3.0
Escacena del Campo,11.0,7.0,10.0,2284.0,306.479860,437.828371,2.0
Cabezas Rubias,2.0,1.0,2.0,730.0,136.986301,273.972603,0.0
Minas de Riotinto,14.0,1.0,9.0,3848.0,25.987526,233.887734,0.0
Isla Cristina,34.0,17.0,22.0,21264.0,79.947329,103.461249,2.0
Palma del Condado (La),29.0,9.0,11.0,10761.0,83.635350,102.220983,5.0
Cortegana,8.0,4.0,4.0,4666.0,85.726532,85.726532,0.0
Bollullos Par del Condado,23.0,11.0,12.0,14272.0,77.073991,84.080717,5.0
Lepe,59.0,8.0,21.0,27431.0,29.164084,76.555722,4.0
